In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [4]:
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
GROQ_API_KEY = os.getenv('GROQ_API_KEY')

In [6]:
os.environ["HUGGINGFACE_API_TOKEN"] = os.getenv("HUGGINGFACE_API_TOKEN")

In [5]:
from langchain_groq import  ChatGroq
model = ChatGroq(model="Gemma2-9b-It")

In [7]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [10]:
import bs4

loader = WebBaseLoader(
    web_paths = ("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs = dict(
        parse_only=bs4.SoupStrainer(
            class_ = ("post-content", "post-title", "post-header")
        )
    )
)

In [12]:
documents = loader.load()

In [15]:
print(documents[0].page_content)



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory

Short-term memory: I 

In [16]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

splits = text_splitter.split_documents(documents)

In [18]:
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [19]:
retriever = vectorstore.as_retriever()

In [20]:
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7f9a26a7aa70>)

In [21]:
# Prompt Template

system_prompt = (
    "You are an assistant for question-answering tasks"
    "Use the following pieces of retrieved context to answer the question"
    "If you don't know the answer , say that you don't know the asnwer."
    "Answer in at moST three sentences."
    "\n\n"
    "{context}"
    
)

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt),
    ("human", "{input}"),]
)

In [22]:
question_answer_chain = create_stuff_documents_chain(model, prompt)

rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [23]:
response = rag_chain.invoke({"input":"What is Self-Reflection"})

In [24]:
response

{'input': 'What is Self-Reflection',
 'context': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 3. Illustration of the Reflexion framework. (Image source: Shinn & Labash, 2023)\nThe heuristic function determines when the trajectory is inefficient or contains hallucination and should be stopped. Inefficient planning refers to trajectories that take too long without success. Hallucination is defined as encountering a sequence of consecutive identical actions that lead to the same observation in the environment.\nSelf-reflection is created by showing two-shot examples to LLM and each example is a pair of (failed trajectory, ideal reflection for guiding future changes in the plan). Then reflections are added into the agent’s working memory, up to three, to be used as context for querying LLM.'),
  Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Another quite distinct approach, LLM

In [26]:
print(response["answer"])

Self-reflection, in the context of the Reflexion framework, is a mechanism that allows agents to improve by analyzing past actions. 

It involves showing the agent examples of failed trajectories and ideal reflections to guide future plan changes. This process helps the agent learn from its mistakes and make better decisions in the future. 





In [31]:
res = rag_chain.invoke({"input":"How do we achieve it?"})

In [32]:
print(res['answer'])

"It" is unclear in your question. Please provide more context about what you are referring to. 


I need to know what "it" is to explain how to achieve it. 



## Adding Chat History

In [33]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

In [34]:
contextualize_q_system_prompt = (
    """
    Given a chat history and the latest user question
    which might reference context in the chat history, 
    fromulate a standalone question which can be understood.
    without chat history, Do not answer the question 
    Just reformulate it if needed and otherwise return it as it is.
    """
)

In [35]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
[
("system", contextualize_q_system_prompt),
MessagesPlaceholder("chat_history"),
("human", "{input}"),
]
)

In [37]:
history_aware_retriever = create_history_aware_retriever(model, retriever, contextualize_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7f9a26a7aa70>))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\n    Given a chat history and the latest user question\n    which might reference context in the chat history, \n    fromulate a standalone question which can be understood.\n    without chat history, Do not answer the question \n

In [39]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
    ("system", system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
    ]
)

In [40]:
question_answer_chain = create_stuff_documents_chain(model, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [41]:
from langchain_core.messages import AIMessage, HumanMessage


chat_history = []
question1 = "What is Self-Reflection"
response1 = rag_chain.invoke({"input":question1, "chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question1),
        AIMessage(content=response1['answer'])
    ]
)

question2 = "Tell me more about it."
response2 = rag_chain.invoke({"input":question2, "chat_history":chat_history})

print(response2['answer'])

Self-reflection in AI agents is a bit like introspection in humans. 

It's the ability to look back at past actions, understand why they were taken, and evaluate their effectiveness. This involves analyzing the agent's own experiences, identifying successes and failures, and using those insights to adjust its future behavior. 

Think of it like a student reviewing their test answers: they identify mistakes, understand the concepts they missed, and use that knowledge to study better for the next exam. 





In [42]:
chat_history

[HumanMessage(content='What is Self-Reflection'),
 AIMessage(content='Self-reflection is a mechanism that helps autonomous agents learn and improve by analyzing past experiences.  \n\nIt involves synthesizing memories into higher-level inferences that can guide future actions. This process allows agents to correct mistakes and refine their plans over time. \n')]

In [43]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key = "answer"
)

In [44]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config = {
        "configurable": {"session_id": "abc123"},
    },
)['answer']

'Task decomposition is the process of breaking down a complex task into smaller, more manageable subtasks.  \n\nThis makes it easier for an agent, like a large language model (LLM), to plan and execute the task effectively. \n  \nLLMs can be prompted to decompose tasks, utilize task-specific instructions, or receive human input for decomposition. \n'

In [45]:
conversational_rag_chain.invoke(
    {'input': 'What are common ways of doing it?'},
    config = {'configurable':{"session_id":"abc123"}},
)['answer']

'Three common ways to perform task decomposition are:\n\n1. **Using simple prompts:**  LLMs can be prompted with phrases like "Steps for XYZ.\\n1." or "What are the subgoals for achieving XYZ?" to guide them in breaking down the task.\n2. **Employing task-specific instructions:** Providing instructions tailored to the specific task, such as "Write a story outline" for writing a novel, can help the LLM decompose the task appropriately. \n3. **Incorporating human input:** Humans can directly provide the initial decomposition of a complex task, giving the LLM a starting point for further planning and execution. \n\n\n\n'